<a href="https://colab.research.google.com/github/Masum06/GenderDebiasing/blob/development/Gender_Encrypter_Decrypter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gender Encryption

### Installation

In [3]:
import spacy
spacy.__version__

'2.1.9'

https://github.com/zalandoresearch/flair

https://github.com/huggingface/neuralcoref

In [4]:
!pip install flair==0.4.2
#!pip uninstall spacy
#!pip install -U spacy==2.1.0
##!python -m spacy download en # SMALL VERSION OF SPACY, FAST DOWNLOAD
##!python -m spacy download en_core_web_md # MEDIUM VERSION OF SPACY
!python -m spacy download en_core_web_lg # LARGE VERSION OF SPACY, SLOW DOWNLOAD
!pip install neuralcoref --no-binary neuralcoref

     |████████████████████████████████| 143kB 10.1MB/s 
     |████████████████████████████████| 798kB 50.1MB/s 
     |████████████████████████████████| 645kB 48.2MB/s 
     |████████████████████████████████| 133kB 47.1MB/s 
     |████████████████████████████████| 1.0MB 50.9MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=fa3d0b2ee053a69f591c6c085a65c38ea0bc802c5fae4a456d97a93dba59d311
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=84f5322466ef66863eba6de99b41527810b49fb79108ef998537cc0386d58cb0
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created wheel for segtok: filename=segtok-1.5.7-cp36-none-any.whl size=23258 sha256=0f17632a935deeefc90dd3eeeff0682e43d74d65d97fea6af5b0b9df3a1ac042
  Stored in directory: /root/.cache/pip/wheels/1

### Creating Language Resources

In [0]:
male_pronouns   = ["he",  "him", "his$", "his", "himself"]
female_pronouns = ["she", "her", "her$", "hers", "herself"]
neutral_pronouns= ["zie", "zim", "zir", "zis", "zieself"]
merged_pronouns = ["he/she", "him/her", "his/her", "his/hers", "himself/herself"]

In [0]:
gender_pronouns_dict = {}
gender_honorific_dict = {}

In [0]:
for (g1,g2,g3,g4) in zip(male_pronouns, female_pronouns, neutral_pronouns, merged_pronouns):
    element = {"male": g1.replace("$",""), "female":g2.replace("$",""), "neutral":g3, "merged":g4}
    gender_pronouns_dict[g1] = gender_pronouns_dict[g2] = gender_pronouns_dict[g3] = gender_pronouns_dict[g4] =element

In [0]:
male_hons   =  ["Mr.", "Mr", "Md.", "Md", "Sir", "Lord", "Mister"]
female_hons =  ["Ms.", "Ms", "Mst.", "Mst", "Madam", "Lady", "Miss"]
neutral_hons = ["Mx.", "Mx", "Mx.", "Mx", "Sir/Madam", "Lord/Lady", "Mister/Miss"]
married_hons = ["Mrs.", "Mrs", "Mst.", "Mst", "Madam", "Lady", "Mis'ess"]
merged_hons =  ["Mr./Ms.", "Mr/Ms", "Md./Mst.", "Md/Mst", "Sir/Madam", "Lord/Lady", "Mister/Miss"]

In [0]:
for (h1,h2,h3,h4,h5) in zip(male_hons, female_hons, neutral_hons, married_hons, merged_hons):
    element = {"male": h1, "female":h2, "neutral":h3, "married_fem":h4, "merged":h5}
    gender_honorific_dict[h1] = gender_honorific_dict[h2] = gender_honorific_dict[h3] = \
    gender_honorific_dict[h4] = gender_honorific_dict[h5] = element

### Imports

In [0]:
import re

In [0]:
from flair.data import Sentence
from flair.models import SequenceTagger
from segtok.segmenter import split_single

In [0]:
import flair
assert flair.__version__=='0.4.2'

In [9]:
from keras.models import load_model

Using TensorFlow backend.


In [10]:
import spacy
import neuralcoref

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)


### Loading Essentials

In [19]:
!git clone --single-branch --branch development https://github.com/Masum06/GenderDebiasing.git resources

fatal: destination path 'resources' already exists and is not an empty directory.


In [0]:
import json

char2idx = 'resources/char2idx.json'
idx2char = 'resources/idx2char.json'
with open(char2idx, 'r') as fp:
    char2idx = json.load(fp)
    
with open(idx2char, 'r') as fp:
    idx2char = json.load(fp)

In [12]:
model_name = 'resources/char_rnn_hsc_model_0.h5'
model = load_model(model_name)

2019-11-07 22:58:47,642 From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

2019-11-07 22:58:47,648 From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

2019-11-07 22:58:47,668 From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

2019-11-07 22:58:47,761 From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

2019-11-07 22:58:47,772 From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated

In [13]:
#nlp = spacy.load('en_core_web_lg') # en_core_web_sm #en_core_web_md
import en_core_web_lg
nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

In [14]:
tagger_ner = SequenceTagger.load('ner')
tagger_pos = SequenceTagger.load('pos')

2019-11-07 22:59:00,804 loading file /root/.flair/models/en-ner-conll03-v0.4.pt
2019-11-07 22:59:02,682 loading file /root/.flair/models/en-pos-ontonotes-v0.2.pt


### Necessary Methods

Name2Gender

In [0]:
import numpy as np
from keras.preprocessing import sequence

# Converts a name into vector
def name2vectorTest(name):
    name = name.lower()
    new_name = ""
    for char in name:
      if char in char2idx:
        new_name += char
    chars = list(new_name)
    vector = [ char2idx[c] for c in chars ]
    return np.array(vector)

# Converts names to fixed size tensor
def names2tensorTest(names, maxlen=25):
    namelist = [name2vectorTest(name) for name in names]
    return sequence.pad_sequences(np.array(namelist), maxlen=maxlen)  # root of all troubles

def name2gender(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  if result:
    return "male"
  else:
    return "female"
  
def isMale(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  return result

Storing Names in Dictionary

In [0]:
def store(name, name_found, Name2Key, Key2Name, num_keys):
    if name_found in Name2Key:
        if name not in Name2Key:
            element = {"name": name, "is_alias": True, "alias_to": name_found}
    else:
        #global num_keys
        num_keys+=1
        key = "PER_"+str(num_keys)
        gender = name2gender(name)
        alias = None
        
        if name!=name_found:
            element_alias = {"name": name, "is_alias": True, "alias_to": name_found}
            Name2Key[name] = element_alias
            alias = name
        
        element = {"name": name, "key": key, "gender":gender, "alias":alias, "is_alias": False, "alias_to": None}
        Name2Key[name_found] = element
        Key2Name[key] = element
        
    return Name2Key[name_found]["key"], num_keys

# Debugging

### Input sentence

In [19]:
s = "Two days before Buet student Abrar was tortured to death, some of his suspects held two meetings at their dorm canteen and guest room and decided to beat him up and evict him if his involvement with Shibir was found."
#s = ' '.join(s.split())
s

'Two days before Buet student Abrar was tortured to death, some of his suspects held two meetings at their dorm canteen and guest room and decided to beat him up and evict him if his involvement with Shibir was found.'

### POS-NER

In [0]:
doc = nlp(s)

In [0]:
oracle = []
coref2name = {}

In [0]:
sent = Sentence(' '.join([token.text for token in doc])) #s, use_tokenizer=True
tagger_ner.predict(sent)
tagger_pos.predict(sent)
tagged_list = sent.to_tagged_string().split()

In [119]:
print(len(sent))

21


In [0]:
tokens_pos = []
pos = []
count = 0
for i in range(0,len(tagged_list),2):
    tokens_pos.append(tagged_list[i])
    count = count+1
    pos.append(tagged_list[i+1])
    
    if tagged_list[i].lower() in male_pronouns+female_pronouns:
        oracle.append(2)
    elif tagged_list[i+1] in ['<B-PER/NNP>', '<I-PER/NNP>', '<E-PER/NNP>', '<S-PER/NNP>']:
        oracle.append(4)
    else:
        oracle.append(0)

### Coreference

In [0]:
assert len(doc)==len(tokens_pos)

In [139]:
doc[1]._.coref_clusters

[]

In [126]:
doc._.coref_clusters[0].mentions[-1]._.coref_cluster.main

Mrs. Katy

**Assumption: The sentence will end with period or any other special token**

In [0]:
coref_stack = []
name_stack = []
for i in range(len(doc)):
    token = doc[i]
    if token._.in_coref:
        coref_stack.append(tokens_pos[i])
        if oracle[i] == 4:
            name_stack.append(tokens_pos[i])
        oracle[i] += 1
    else:
        if len(name_stack) > 0:
            name = ' '.join(name_stack)
            coref = ' '.join(coref_stack)
            #name2coref[name] = coref
            coref2name[coref] = name
            name_stack.clear()
        coref_stack.clear()

# IF THE SENTENCE DOES NOT END WITH A PERIOD OR SPECIAL CHARACTER
if len(name_stack) > 0:
    name = ' '.join(name_stack)
    name2coref[name] = ' '.join(coref_stack)
    name_stack.clear()
coref_stack.clear()

In [0]:
Name2Key = {}
Key2Name = {}
encrypted = []
num_keys = 0

i = 0
while i<len(tokens_pos): 
    #print("Oracle ", i, tokens_pos[i])
    if oracle[i] == 0:
        encrypted.append(tokens_pos[i])
    elif oracle[i+1] in [4,5]:
        if tokens_pos[i] in gender_honorific_dict:
            encrypted.append("<|hons|>")
            encrypted.append(gender_honorific_dict[tokens_pos[i]]["merged"])
        else:
            encrypted.append(tokens_pos[i])
    elif oracle[i] == 2:
        pronoun = tokens_pos[i].lower()
        if pos[i] == '<PRP$>':
            pronoun+="$"
        encrypted.append(gender_pronouns_dict[pronoun]["merged"])
    elif oracle[i] == 3:
        coref = doc[i]._.coref_clusters[0][0].text
        pronoun = tokens_pos[i].lower()
        if pos[i] == '<PRP$>':
            pronoun+="$"
        if coref in coref2name:
          name_found = coref2name[coref]
          key = Name2Key[name_found]["key"]
          encrypted.append("<|coref|>")
          encrypted.append(gender_pronouns_dict[pronoun]["merged"])
          encrypted.append(key)
        else:
          encrypted.append(gender_pronouns_dict[pronoun]["merged"])
      
    elif oracle[i] in [4,5]:
        if pos[i] == '<S-PER/NNP>':
            name = tokens_pos[i]
        elif pos[i] == '<B-PER/NNP>':
            name = ""
            while True:
                print(i, oracle[i])
                name += tokens_pos[i]
                if pos[i] == '<E-PER/NNP>':
                    break
                name += " "
                i+=1
        
        if oracle[i] == 4:
            key, num_keys = store(name, name, Name2Key, Key2Name, num_keys)
            encrypted.append(key)
        else:
            coref = doc[i]._.coref_clusters[0][0].text
            name_found = coref2name[coref]
            if name == name_found:
                key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                encrypted.append(key)
            else:
                key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                encrypted.append("<|alias|>")
                encrypted.append(key)
    i+=1

In [113]:
for x,y in zip(tokens_pos, oracle):
  print(x,y)

Mr. 0
Masum 4
is 0
not 0
a 0
doctor 0
. 0
Mrs. 1
Katy 5
is 0
also 0
not 0
an 1
engineer 1
. 0
He 3
can 0
fly 0
, 0
but 0
she 3
can 0
not 0
swim 0
. 0


In [114]:
doc._.coref_clusters

[Mrs. Katy: [Mrs. Katy, she], an engineer: [an engineer, He]]

In [0]:
coref2name

{'Two days before Buet student Abrar was tortured to death': 'Abrar'}

In [0]:
s

'Two days before Buet student Abrar was tortured to death, some of the suspects held two meetings at their dorm canteen and guest room and decided to beat him up and evict him if his involvement with Shibir was found.'

In [0]:
encrypted

['Two',
 'days',
 'before',
 'Buet',
 'student',
 'PER_1',
 'was',
 'tortured',
 'to',
 'death',
 ',',
 'some',
 'of',
 '<|coref|>',
 'his/her',
 'PER_1',
 'suspects',
 'held',
 'two',
 'meetings',
 'at',
 'their',
 'dorm',
 'canteen',
 'and',
 'guest',
 'room',
 'and',
 'decided',
 'to',
 'beat',
 '<|coref|>',
 'him/her',
 'PER_1',
 'up',
 'and',
 'evict',
 '<|coref|>',
 'him/her',
 'PER_1',
 'if',
 '<|coref|>',
 'his/her',
 'PER_1',
 'involvement',
 'with',
 'PER_2',
 'was',
 'found',
 '.']

In [0]:
Name2Key

In [0]:
Key2Name

{'PER_1': {'name': 'Abrar',
  'key': 'PER_1',
  'gender': 'Male',
  'alias': None,
  'is_alias': False,
  'alias_to': None},
 'PER_2': {'name': 'Shibir',
  'key': 'PER_2',
  'gender': 'Male',
  'alias': None,
  'is_alias': False,
  'alias_to': None}}

In [0]:
encrypted

['Two',
 'days',
 'before',
 'Buet',
 'student',
 'PER_1',
 'was',
 'tortured',
 'to',
 'death',
 ',',
 'some',
 'of',
 '<|coref|>',
 'his/her',
 'PER_1',
 'suspects',
 'held',
 'two',
 'meetings',
 'at',
 'their',
 'dorm',
 'canteen',
 'and',
 'guest',
 'room',
 'and',
 'decided',
 'to',
 'beat',
 '<|coref|>',
 'him/her',
 'PER_1',
 'up',
 'and',
 'evict',
 '<|coref|>',
 'him/her',
 'PER_1',
 'if',
 '<|coref|>',
 'his/her',
 'PER_1',
 'involvement',
 'with',
 'PER_2',
 'was',
 'found',
 '.']

# Unit Gender Encryption

In [0]:
def gender_encrypt(s):
  s = ' '.join(s.split())
  doc = nlp(s)
  tokenized_text = ' '.join([token.text for token in doc])
  oracle = []
  coref2name = {}

  # POS TAG
  sent = Sentence(tokenized_text)
  tagger_ner.predict(sent)
  tagger_pos.predict(sent)
  tagged_list = sent.to_tagged_string().split()
  tokens_pos = []
  pos = []
  count = 0
  for i in range(0,len(tagged_list),2):
      tokens_pos.append(tagged_list[i])
      count = count+1
      pos.append(tagged_list[i+1])
      
      if tagged_list[i].lower() in male_pronouns+female_pronouns:
          oracle.append(2)
      elif tagged_list[i+1] in ['<B-PER/NNP>', '<I-PER/NNP>', '<E-PER/NNP>', '<S-PER/NNP>']:
          oracle.append(4)
      else:
          oracle.append(0)

  # COREFERENCE RESOLUTION 
  if len(doc)!=len(tokens_pos):
    tokens_doc = [token.text for token in doc]
    print("doc:", tokens_doc)
    print("pos:", tokens_pos)
    return None, None
  
  coref_stack = []
  name_stack = []
  for i in range(len(doc)):
      token = doc[i]
      if token._.in_coref:
          coref_stack.append(tokens_pos[i])
          if oracle[i] == 4:
              name_stack.append(tokens_pos[i])
          oracle[i] += 1
      else:
          if len(name_stack) > 0:
              name = ' '.join(name_stack)
              coref = ' '.join(coref_stack)
              #name2coref[name] = coref
              coref2name[coref] = name
              name_stack.clear()
          coref_stack.clear()

  # IF THE SENTENCE DOES NOT END WITH A PERIOD OR SPECIAL CHARACTER
  if len(name_stack) > 0:
      name = ' '.join(name_stack)
      name2coref[name] = ' '.join(coref_stack)
      name_stack.clear()
  coref_stack.clear()

  Name2Key = {}
  Key2Name = {}
  encrypted = []
  num_keys = 0
  i = 0
  while i<len(tokens_pos): 
      #print("Oracle ", i, tokens_pos[i])
      if oracle[i] == 2:
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          encrypted.append(gender_pronouns_dict[pronoun]["merged"])
      elif oracle[i] == 3:
          coref = doc[i]._.coref_clusters[0][0].text
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          if coref in coref2name:
            name_found = coref2name[coref]
            key = Name2Key[name_found]["key"]
            encrypted.append("<|coref|>")
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
            encrypted.append(key)
          else:
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
          
      elif oracle[i] in [4,5]:
          if i > 0 and tokens_pos[i-1] in gender_honorific_dict:
            hons = encrypted.pop()
            encrypted.append("<|hons|>")
            encrypted.append(gender_honorific_dict[hons]["merged"])
          if pos[i] == '<S-PER/NNP>':
              name = tokens_pos[i]
          elif pos[i] == '<B-PER/NNP>':
              name = ""
              while True:
                  print(i, oracle[i])
                  name += tokens_pos[i]
                  if pos[i] == '<E-PER/NNP>':
                      break
                  name += " "
                  i+=1
          
          if oracle[i] == 4:
              key, num_keys = store(name, name, Name2Key, Key2Name, num_keys)
              encrypted.append(key)
          else:
              coref = doc[i]._.coref_clusters[0][0].text
              name_found = coref2name[coref]
              if name == name_found:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append(key)
              else:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append("<|alias|>")
                  encrypted.append(key)
      else:
        encrypted.append(tokens_pos[i])
      i+=1
      
  encrypted_text = ' '.join(encrypted)

  return tokenized_text, Key2Name, encrypted_text

# Gender Decryption

In [0]:
def gender_decrypt(encrypted_text, Key2Name):
  encrypted = encrypted_text.split()
  decrypted = []
  i = 0
  while i != len(encrypted):
    token = encrypted[i]
    if token in Key2Name:
      decrypted.append(Key2Name[token]["name"])
    elif token == '<|alias|>':
      i+=1
      key = encrypted[i]
      if Key2Name[key]["alias"]:
        decrypted.append(Key2Name[key]["alias"])
      else:
        decrypted.append(Key2Name[key]["name"])
    elif token == '<|coref|>':
      pronoun =  encrypted[i+1]
      key = encrypted[i+2]
      gender = Key2Name[key]["gender"].lower()
      decrypted.append(gender_pronouns_dict[pronoun][gender])
      i+=2
    elif token == "<|hons|>":
      hons = encrypted[i+1]
      key = encrypted[i+2] # NEED CHECK FOR HONS IN ALIAS
      gender = Key2Name[key]["gender"].lower()
      decrypted.append(gender_honorific_dict[hons][gender])
      i+=1
    else:
      decrypted.append(token)
    i+=1

  decrypted_text = ' '.join(decrypted)
  #decrypted_text = decrypted_text.replace(" .", ".")
  return decrypted_text

In [0]:
key2name_changed = {'PER_1': {'name': 'Abira',
    'key': 'PER_1',
    'gender': 'Female',
    'alias': None,
    'is_alias': False,
    'alias_to': None},
  'PER_2': {'name': 'Shibir',
    'key': 'PER_2',
    'gender': 'Male',
    'alias': None,
    'is_alias': False,
    'alias_to': None}
  }

### Testing Encryption-Decryption

In [0]:
s = "Mr. John is not a doctor. Ms. Katy is also not a nurse. He can fly, but she cannot swim."

In [0]:
s = "Morshed Khan told newsmen that 'BNP is a massive party with huge popularity and public acceptance.'"# This party is now running its operations over Skype. He (Tarique) is operating from London over Skype and that too with only selected leaders. It has turned into the ‘Bangladesh Nationalist Skype Party’. This is painful. I believe the next generation must assume leadership within the party — that would be my recommendation.”"

In [20]:
len(s.split())

16

In [0]:
import time 
start = time.time()
text, key2name, encrypted = gender_encrypt(s)
end = time.time()

print(encrypted)
print(key2name)
print(end-start)

In [0]:
key2name = {'PER_1': {'name': 'Masum', 'key': 'PER_1', 'gender': 'female', 'alias': None, 'is_alias': False, 'alias_to': None}}

In [41]:
decrypted = gender_decrypt(encrypted, key2name)
decrypted

'Prime Minister Sheikh Hasina on Wednesday reiterated her government ’s stance on not setting up industries by destroying arable land . “ We have set up 100 economic zones so that the arable land is not destroyed . Those , who want to set up industries , will be given plots in the zones . It is because we have to save arable land , ” she said , adding that farmers are given the highest importance while undertaking any development programme .'

In [42]:
decrypted == text

True

In [35]:
en = gender_encrypt("Mr. Masum is not a doctor.")
en

('<|hons|> Mr./Ms. PER_1 is not a doctor .',
 {'PER_1': {'alias': None,
   'alias_to': None,
   'gender': 'male',
   'is_alias': False,
   'key': 'PER_1',
   'name': 'Masum'}})